# 串接雲端
資料需要事先傳到雲端，並執行下面程式碼，點選該雲端，複製驗證碼後，貼上執行後出現的方匡內，即可驗證完畢。

第一次執行時間需等待較久，為正常情況。

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 載入函示庫

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt

# 模型說明
Model 11 = 適當的控制變數  Core_Technological_Coherence  Core_Technological_Coherence^2 Technological_distance  Core_Technological_Coherence* Technological_distance Core_Technological_Coherence^2* Technological_distance

Y 1 = forward citation ranking_prior 5 years(fixed effect adjust) top 3%


***注意事項***
1. 資料需要事先存到雲端
2. 資料需存為xlsx檔
3. 資料標頭要小心是否有空格，建議一率從excel複製，不要自己打
4. 資料若有缺失值，需統一將缺失值都設定為***，可以只修改會用到的變數即可
5. 全部程式碼只需更動下面這一個，將相關設定成自己的，其他都不需要更動，執行就好！


In [134]:
# 設定資料（唯一需要修改的地方）

# 回歸：控制變數＆Ｘ
var = ['Ln_Firm_age ','Ln_firm_size(emp)',' R&D_intensity(xrd/ sale) ','Ln number of alliances','Ln Patent stock','CTC_edited','CTC^2','technological distance_edited','CTC*Technology distance','CTC^2*Technology distance']
# 回歸：Ｙ
DV = 'forward citation ranking_prior 5 years(fixed effect adjust) top 3%'
# 讀入excel檔案的路徑
input_path = r"/content/drive/MyDrive/data_MS/資料總檔/新財務變數/CTC模型/excel檔案/telecommunication_0826.xlsx"
# 匯出excel檔案的路徑
output_path = r"/content/drive/MyDrive/data_MS/資料總檔/新財務變數/CTC模型/excel檔案/del_telecommunication_0826.xlsx"

In [135]:
# 載入存在雲端的資料（xlsx檔）
df = pd.read_excel(input_path, keep_default_na=False) 

In [136]:
# 確認資料是否正確
df

,Company,CompanyType,Year,founded year,cusip-6,SIC,Unnamed: 6,Unnamed: 7,Unnamed: 8,age,gvkey,datadate,fyear,tic,cusip,Unnamed: 15,conm,at,bkvlps,capx,cogs,cshi,csho,dltt,dt,emp,ni,niadj,ppent,pstkl,rdip,revt,sale,xad,xagt,xint,xopr,xoprar,xrd,xrdp,...,competition2_dummy_edited * Cooperation2_dummy * Cost leadership,competition2_dummy_edited * Differentiation,Cooperation2_dummy * Differentiation,competition2_dummy_edited * Cooperation2_dummy * Differentiation,(Competition1 + Cooperation1) * Cost leadership,(Competition1 + Cooperation1)_square * Cost leadership,(Competition1 + Cooperation1) * Differentiation,(Competition1 + Cooperation1)_square * Differentiation,Coopetition_dummy * Cost leadership,Coopetition_dummy * Differentiation,technology similarity_edited * Cost leadership,technology similarity_square_edited* Cost leadership,technology similarity_edited * repeated tie_edited * Cost leadership,technology similarity_edited * repeated tie_square * Cost leadership,technology similarity_square_edited * repeated tie_edited * Cost leadership,technology similarity_edited * Differentiation,technology similarity_square_edited* Differentiation,technology similarity_edited * repeated tie_edited * Differentiation,technology similarity_edited * repeated tie_square * Differentiation,technology similarity_square_edited * repeated tie_edited * Differentiation,Participant composition_new_edited* Cost leadership,Participant composition_new_edited_square* Cost leadership,repeated tie_edited * Cost leadership,repeated tie_square * Cost leadership,Participant composition_new_edited * repeated tie_edited* Cost leadership,Participant composition_new_edited * repeated tie_square* Cost leadership,Participant composition_new_edited_square * repeated tie_edited* Cost leadership,Participant composition_new_edited* Differentiation,Participant composition_new_edited_square* Differentiation,repeated tie_edited * Differentiation,repeated tie_square * Differentiation,Participant composition_new_edited * repeated tie_edited* Differentiation,Participant composition_new_edited * repeated tie_square* Differentiation,Participant composition_new_edited_square * repeated tie_edited* Differentiation,technology_similarity_dummy* Cost leadership,repeated tie_dummy* Cost leadership,technology_similarity_dummy* Differentiation,repeated tie_dummy* Differentiation,participation_dummy * Cost leadership,participation_dummy * Differentiation
0,1,55551,1996,1996,282914,4899,,,,0,65021,30469,1995,EGHT,282914100,,8X8 INC,12.727,-1.1522,0.691,11.164,,6.99,0,,0.098,-13.613,-13.613,1.344,14.095,,19.146,19.146,,,0,31.772,,10.51,7.714,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,55551,1997,1996,282914,4899,,,,1,65021,30469,1996,EGHT,282914100,,8X8 INC,46.429,2.3828,0.927,17.95,,15.294,0,,0.136,3.727,3.727,1.37,0,,49.776,49.776,,,0,47.648,,12.317,10.51,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1,55551,1998,1996,282914,4899,,,,2,65021,30469,1997,EGHT,282914100,,8X8 INC,28.709,1.2202,1.76,17.614,15.426,15.426,0,0,0.146,-19.224,-19.224,2.163,0,,31.682,31.682,,,0,45.248,,9.922,12.317,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1,55551,1999,1996,282914,4899,,,,3,65021,30469,1998,EGHT,282914100,,8X8 INC,59.983,2.0641,1.693,7.926,22.959,22.959,5.498,5.498,0.147,-24.848,-24.848,2.687,0,,25.384,25.384,,,0.391,34.742,,22.009,9.922,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1,55551,2000,1996,282914,4899,,,,4,65021,30469,1999,EGHT,282914100,,8X8 INC,39.145,0.8108,6.127,3.618,26.681,26.681,6.238,6.238,0.156,-74.399,-74.399,5.016,0,,18.228,18.228,,,1.456,40.467,,23.299,22.009,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9248,452,5555452,2012,1977,98978K,3661,,,,

In [137]:
# 刪除含有缺失值的列（缺失值需統一設定成***）
df_isNA = df
for v in var:
  df_isNA = df_isNA.loc[df_isNA[v] != "***" ]

df_isNA = df_isNA.reset_index(drop=True)

In [138]:
# 確認資料是否正確刪除含缺失值的列
df_isNA 

,Company,CompanyType,Year,founded year,cusip-6,SIC,Unnamed: 6,Unnamed: 7,Unnamed: 8,age,gvkey,datadate,fyear,tic,cusip,Unnamed: 15,conm,at,bkvlps,capx,cogs,cshi,csho,dltt,dt,emp,ni,niadj,ppent,pstkl,rdip,revt,sale,xad,xagt,xint,xopr,xoprar,xrd,xrdp,...,competition2_dummy_edited * Cooperation2_dummy * Cost leadership,competition2_dummy_edited * Differentiation,Cooperation2_dummy * Differentiation,competition2_dummy_edited * Cooperation2_dummy * Differentiation,(Competition1 + Cooperation1) * Cost leadership,(Competition1 + Cooperation1)_square * Cost leadership,(Competition1 + Cooperation1) * Differentiation,(Competition1 + Cooperation1)_square * Differentiation,Coopetition_dummy * Cost leadership,Coopetition_dummy * Differentiation,technology similarity_edited * Cost leadership,technology similarity_square_edited* Cost leadership,technology similarity_edited * repeated tie_edited * Cost leadership,technology similarity_edited * repeated tie_square * Cost leadership,technology similarity_square_edited * repeated tie_edited * Cost leadership,technology similarity_edited * Differentiation,technology similarity_square_edited* Differentiation,technology similarity_edited * repeated tie_edited * Differentiation,technology similarity_edited * repeated tie_square * Differentiation,technology similarity_square_edited * repeated tie_edited * Differentiation,Participant composition_new_edited* Cost leadership,Participant composition_new_edited_square* Cost leadership,repeated tie_edited * Cost leadership,repeated tie_square * Cost leadership,Participant composition_new_edited * repeated tie_edited* Cost leadership,Participant composition_new_edited * repeated tie_square* Cost leadership,Participant composition_new_edited_square * repeated tie_edited* Cost leadership,Participant composition_new_edited* Differentiation,Participant composition_new_edited_square* Differentiation,repeated tie_edited * Differentiation,repeated tie_square * Differentiation,Participant composition_new_edited * repeated tie_edited* Differentiation,Participant composition_new_edited * repeated tie_square* Differentiation,Participant composition_new_edited_square * repeated tie_edited* Differentiation,technology_similarity_dummy* Cost leadership,repeated tie_dummy* Cost leadership,technology_similarity_dummy* Differentiation,repeated tie_dummy* Differentiation,participation_dummy * Cost leadership,participation_dummy * Differentiation
0,1,55551,2004,1996,282914,4899,,,,8,65021,30469,2003,EGHT,282914100,,8X8 INC,39.08,0.5527,1.588,9.526,53.817,53.817,0,0,0.093,-19.148,-19.148,1.788,0,0,11.475,11.475,4.1,,0,31.169,,3.109,2.747,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,55551,2005,1996,282914,4899,,,,9,65021,30470,2004,EGHT,282914100,,8X8 INC,31.12,0.3286,1.9,22.312,61.138,61.138,0.07,,0.139,-24.139,-24.139,3.071,0,0,31.892,31.892,5.265,,,56.091,,5.916,3.109,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1,55551,2006,1996,282914,4899,,,,10,65021,30470,2005,EGHT,282914100,,8X8 INC,19.958,0.087,1.424,25.664,61.772,61.772,0.122,0.122,0.16,-9.93,-9.93,2.84,0,0,53.13,53.13,5.614,,,66.033,,4.712,5.916,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1,55551,2007,1996,282914,4899,,,,11,65021,30470,2006,EGHT,282914100,,8X8 INC,21.551,0.1265,0.699,21.904,62.067,62.067,0.086,0.122,0.197,0.03,0.03,2.01,0,0,61.073,61.073,6.989,,,63.835,,4.335,4.712,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1,55551,2008,1996,282914,4899,,,,12,65021,30470,2007,EGHT,282914100,,8X8 INC,21.856,0.1441,0.801,21.58,62.686,62.686,0.046,0.085,0.244,-2.5,-2.5,1.485,0,0,64.674,64.674,7.297,,,66.472,,5.212,4.335,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,447,5555447,2003,1988,

# 多元回歸，刪除outliers

執行多元回歸，每一次回歸刪除最大殘差的觀察值，然後再次執行，直到沒有outliers

In [139]:
# 刪掉觀察值的數量
count = 0

# 執行多元回歸
while True:

  # 設定回歸的Ｘ& Y
  X = df_isNA[var]
  y = df_isNA[DV].values
  #print(X.shape, y.shape)
  
  # 多元回歸
  X2 = sm.add_constant(X) 
  res = sm.OLS(y, X2.astype(float)).fit()
  beta = res.params
  #print(res.summary()) #總回歸分析與檢驗
  #print(res.params) #beta值  

  # 檢查Beta值，如果beta = 0，則不能做回歸，實驗結束。
  sum_beta = 0
  for i in range(len(beta)):
    sum_beta = sum_beta+beta[i]
  if sum_beta==0:
    print("all beta = 0", "實驗結束！")
    break

  # 計算每筆觀察值的標準殘差，以利分辨outlier
  y_array = y.ravel()
  #估計值
  predict = res.predict() 
  predict_array = np.array(predict)
  #殘差
  subtract = y_array-predict_array
  #標準化估計 ((Σ(y-y')^2)/n)^1/2
  std_predict = sqrt(sum(np.square(subtract))/len(y_array)) #殘差
  #標準殘差 = 殘差/標準化估計值
  std_subtract = subtract/std_predict

  # 取最大的標準殘差在第幾筆
  std_subtract_list = std_subtract.tolist()
  max_std_index = std_subtract_list.index(max(std_subtract_list))

  # 確認是否有標準殘差>3的觀察值
  num_tempOutlier = 0
  #如果有，則 num_tempOutlier > 0
  for i in range(len(std_subtract)):
    if std_subtract[i]>3.5:
      num_tempOutlier = num_tempOutlier + 1

  # 如果有，則刪除標準殘差最大的那列觀察值
  if num_tempOutlier > 0:
    df_isNA = df_isNA.drop([max_std_index],axis=0)
    df_isNA= df_isNA.reset_index(drop=True)
    count = count+1
  # 如果沒有，則實驗結束
  else:
    break

# 確認成功刪除outliers

In [140]:
# 確認刪除後的X&y的大小
X.shape, y.shape

((1156, 10), (1156,))

In [141]:
#確認刪除多少個outliers
count

182

#匯出excel檔案

In [142]:
# 儲存excel檔案
df_isNA.to_excel(output_path, header=True)